In [1]:
import torch
import cv2
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pickle

In [2]:
os.chdir("/Users/eugenekim/PycharmProjects/asl-alphabet-classification/data")

In [3]:
numerical_map = {"A":0, "B":1, "C":2, "D":3, "del":4, "E":5, "F":6,
                "G":7,"H":8,"I":9,"J":10,"K":11,"L":12,"M":13,"N":14,
                "nothing":15,"O":16,"P":17,"Q":18,"R":19,"S":20,
                 "space":21,"T":22,"U":23,"V":24,"W":25,"X":26,"Y":27,"Z":28}

In [91]:
file_path = "./raw_data"
test_path = "asl_alphabet_test"
train_path = "asl_alphabet_train"

transform = transforms.Compose([transforms.ToTensor()])

def get_asl_data(num_image, split="train", normalized=False):
    
    #Desired parameters
    inputs = "empty"
    
    if split == "test":
        
        outputs = None
                
        for letter in sorted(os.listdir(os.path.join(file_path,test_path))):

            #Check if image ends with ".jpg"
            if not letter.endswith(".jpg"):
                continue

            #Read image
            read_image = cv2.imread(os.path.join(file_path,test_path,letter))

            #Convert to RGB
            temp_image = cv2.cvtColor(read_image, cv2.COLOR_BGR2RGB)
                
            #Convert the image to tensor
            image_tensor = transform(temp_image)
                
            if inputs == "empty":
                inputs = image_tensor
                    
            else:
                inputs = torch.vstack((inputs,image_tensor))
        
        return inputs, outputs
    
    if split == "train":
            
        #Desired labels
        outputs = "empty"
            
        for letter in sorted(os.listdir(os.path.join(file_path,train_path))):
            
            count = 0
                
            if letter == ".DS_Store":
                continue
                    
            map_letter = numerical_map[letter]
                
            for image in sorted(os.listdir(os.path.join(file_path,train_path,letter))):
                
                if count == num_image:
                    break
                
                if not image.endswith(".jpg"):
                    continue

                read_image = cv2.imread(os.path.join(file_path,train_path,letter,image))

                #Convert to RGB
                temp_image = cv2.cvtColor(read_image, cv2.COLOR_BGR2RGB)
                    
                #Convert the image to tensor
                image_tensor = transform(temp_image)
                
                if outputs == "empty":
                    outputs = torch.tensor([map_letter])
                    
                else:
                    outputs = torch.vstack((outputs,torch.tensor([map_letter])))
                
                if inputs == "empty":
                    inputs = image_tensor
                    count += 1
                    
                else:
                    inputs = torch.vstack((inputs,image_tensor))
                    count += 1
            
        return inputs, outputs
                    

In [92]:
class ASLDataset(Dataset):
    
    def __init__(self, split, num_image, transform=None):
        
        self.transform = transform
        self.inputs, self.outputs = get_asl_data(num_image, split = split, normalized=False)
        
    def __len__(self):
        
        return len(self.inputs)
    
    def __getitem__(self, idx):

        data = (self.inputs[idx], self.outputs[idx])
        
        if self.transform:
            data = self.transform(data)
            
        return data

In [102]:
train_dataset = ASLDataset(split = "train", num_image=50)

In [104]:
torch.save(train_dataset, "ASLDataset.pkl")